## Вграждания

В предишния пример работихме с високодименсионални вектори на чанта с думи с дължина `vocab_size`, и изрично преобразувахме нискодименсионалните вектори на позиционно представяне в редки едноточкови представяния. Това едноточково представяне не е ефективно по отношение на паметта. Освен това, всяка дума се третира независимо от другите, така че едноточково кодираните вектори не изразяват семантични прилики между думите.

В този модул ще продължим да изследваме набора от данни **News AG**. За начало, нека заредим данните и вземем някои дефиниции от предишния модул.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Какво е embedding?

Идеята на **embedding** е да представим думите чрез нискоразмерни плътни вектори, които отразяват семантичното значение на думата. По-късно ще обсъдим как да създадем смислени word embeddings, но засега нека просто мислим за embedding като начин за намаляване на размерността на вектор на дума.

Така че, embedding слой приема дума като вход и произвежда изходен вектор със зададен размер `embedding_size`. В известен смисъл, той е много подобен на слой `Dense`, но вместо да приема вектор с one-hot кодиране като вход, той може да приема номер на дума.

Като използваме embedding слой като първи слой в нашата мрежа, можем да преминем от модел bag-of-words към модел **embedding bag**, където първо преобразуваме всяка дума в текста в съответния embedding, а след това изчисляваме някаква агрегираща функция върху всички тези embeddings, като например `sum`, `average` или `max`.

![Изображение, показващо embedding класификатор за пет последователни думи.](../../../../../translated_images/bg/embedding-classifier-example.b77f021a7ee67eee.webp)

Нашата невронна мрежа за класификация се състои от следните слоеве:

* Слой `TextVectorization`, който приема низ като вход и произвежда тензор от номера на токени. Ще зададем разумен размер на речника `vocab_size` и ще игнорираме по-рядко използваните думи. Формата на входа ще бъде 1, а формата на изхода ще бъде $n$, тъй като ще получим $n$ токени като резултат, всеки от тях съдържащ числа от 0 до `vocab_size`.
* Слой `Embedding`, който приема $n$ числа и намалява всяко число до плътен вектор с дадена дължина (100 в нашия пример). Така входният тензор с форма $n$ ще бъде трансформиран в тензор с форма $n\times 100$.
* Агрегиращ слой, който взема средната стойност на този тензор по първата ос, т.е. ще изчисли средната стойност на всички $n$ входни тензори, съответстващи на различни думи. За да реализираме този слой, ще използваме слой `Lambda` и ще му предадем функцията за изчисляване на средната стойност. Изходът ще има форма 100 и ще бъде численото представяне на цялата входна последователност.
* Финален `Dense` линеен класификатор.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


В `summary` таблицата, в колоната **output shape**, първото измерение на тензора `None` съответства на размера на минипартидата, а второто - на дължината на последователността от токени. Всички последователности от токени в минипартидата имат различни дължини. Ще обсъдим как да се справим с това в следващия раздел.

Сега нека обучим мрежата:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Забележка**: изграждаме векторизатор, базиран на подмножество от данните. Това се прави с цел ускоряване на процеса и може да доведе до ситуация, в която не всички токени от нашия текст присъстват в речника. В този случай тези токени ще бъдат игнорирани, което може да доведе до леко по-ниска точност. Въпреки това, в реалния живот подмножество от текста често дава добра оценка на речника.


### Работа с променливи размери на последователности

Нека разберем как се извършва обучението в минипартиди. В горния пример входният тензор има размерност 1 и използваме минипартиди с дължина 128, така че реалният размер на тензора е $128 \times 1$. Въпреки това, броят на токените във всяко изречение е различен. Ако приложим слоя `TextVectorization` към един вход, броят на върнатите токени ще бъде различен в зависимост от начина, по който текстът е токенизиран:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Въпреки това, когато приложим векторизатора към няколко последователности, той трябва да произведе тензор с правоъгълна форма, затова запълва неизползваните елементи с токена PAD (който в нашия случай е нула):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Тук можем да видим вгражданията:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Забележка**: За да се минимизира количеството на допълнителните символи, в някои случаи има смисъл да се сортират всички последователности в набора от данни по реда на нарастваща дължина (или, по-точно, брой токени). Това ще гарантира, че всяка минипартида съдържа последователности с подобна дължина.


## Семантични вграждания: Word2Vec

В предишния пример слоят за вграждане научи как да преобразува думите във векторни представяния, но тези представяния нямаха семантично значение. Би било полезно да се научи векторно представяне, при което подобни думи или синоними съответстват на вектори, които са близо един до друг според някакво векторно разстояние (например евклидово разстояние).

За да постигнем това, трябва предварително да обучим модела за вграждане върху голяма колекция от текстове, използвайки техника като [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Тя се основава на две основни архитектури, които се използват за създаване на разпределено представяне на думите:

 - **Непрекъсната торба с думи** (CBoW), при която обучаваме модела да предсказва дума въз основа на заобикалящия контекст. Даден е n-грам $(W_{-2},W_{-1},W_0,W_1,W_2)$, като целта на модела е да предскаже $W_0$ от $(W_{-2},W_{-1},W_1,W_2)$.
 - **Непрекъснат скип-грам** е противоположен на CBoW. Моделът използва заобикалящия прозорец от контекстни думи, за да предскаже текущата дума.

CBoW е по-бърз, докато скип-грам е по-бавен, но се справя по-добре с представянето на редки думи.

![Изображение, показващо алгоритмите CBoW и Skip-Gram за преобразуване на думи във вектори.](../../../../../translated_images/bg/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

За да експериментираме с вграждането Word2Vec, предварително обучено върху набора от данни Google News, можем да използваме библиотеката **gensim**. По-долу намираме думите, които са най-близки до 'neural'.

> **Забележка:** Когато за първи път създавате векторите на думите, изтеглянето им може да отнеме известно време!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Можем също така да извлечем векторното вграждане от думата, за да го използваме при обучението на модела за класификация. Вграждането има 300 компонента, но тук показваме само първите 20 компонента на вектора за яснота:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Страхотното при семантичните вграждания е, че можете да манипулирате векторното кодиране въз основа на семантиката. Например, можем да поискаме да намерим дума, чиято векторна репрезентация е възможно най-близка до думите *крал* и *жена*, и възможно най-далеч от думата *мъж*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Примерът по-горе използва малко вътрешна магия на GenSym, но основната логика всъщност е доста проста. Интересното при вгражданията е, че можете да извършвате нормални операции с вектори върху векторите на вгражданията, и това би отразило операции върху **значенията** на думите. Примерът по-горе може да бъде изразен чрез операции с вектори: изчисляваме вектора, съответстващ на **KING-MAN+WOMAN** (операциите `+` и `-` се извършват върху векторните представяния на съответните думи), и след това намираме най-близката дума в речника до този вектор:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Трябваше да добавим малки коефициенти към векторите на *man* и *woman* - опитайте да ги премахнете, за да видите какво ще се случи.

За да намерим най-близкия вектор, използваме механизма на TensorFlow, за да изчислим вектор от разстояния между нашия вектор и всички вектори в речника, а след това намираме индекса на минималната дума, използвайки `argmin`.


Докато Word2Vec изглежда като чудесен начин за изразяване на семантиката на думите, той има много недостатъци, включително следните:

* И двата модела CBoW и skip-gram са **предсказващи вграждания**, и те вземат предвид само локалния контекст. Word2Vec не използва глобалния контекст.
* Word2Vec не отчита **морфологията** на думите, т.е. факта, че значението на думата може да зависи от различни части на думата, като корена.

**FastText** се опитва да преодолее второто ограничение и надгражда Word2Vec, като учи векторни представяния за всяка дума и n-грамите от символи, които се намират в рамките на всяка дума. Стойностите на тези представяния след това се осредняват в един вектор на всяка стъпка от обучението. Въпреки че това добавя много допълнителни изчисления към предварителното обучение, то позволява вгражданията на думите да кодират информация за поддуми.

Друг метод, **GloVe**, използва различен подход към вгражданията на думи, базиран на факторизацията на матрицата на контекста на думите. Първо, той изгражда голяма матрица, която брои броя на срещанията на думите в различни контексти, и след това се опитва да представи тази матрица в по-ниски измерения по начин, който минимизира загубата при реконструкция.

Библиотеката gensim поддържа тези вграждания на думи, и можете да експериментирате с тях, като промените кода за зареждане на модела по-горе.


## Използване на предварително обучени вграждания в Keras

Можем да модифицираме горния пример, за да запълним предварително матрицата в нашия слой за вграждане със семантични вграждания, като Word2Vec. Речниците на предварително обученото вграждане и текстовия корпус вероятно няма да съвпадат, така че трябва да изберем един от тях. Тук разглеждаме двете възможни опции: използване на речника на токенизатора и използване на речника от Word2Vec вгражданията.

### Използване на речника на токенизатора

Когато използваме речника на токенизатора, някои от думите в речника ще имат съответстващи Word2Vec вграждания, а други ще липсват. Като се има предвид, че размерът на нашия речник е `vocab_size`, а дължината на векторите на Word2Vec вгражданията е `embed_size`, слоят за вграждане ще бъде представен чрез тегловна матрица с форма `vocab_size`$\times$`embed_size`. Ще запълним тази матрица, като преминем през речника:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


За думи, които не присъстват в Word2Vec речника, можем или да ги оставим като нули, или да генерираме случаен вектор.

Сега можем да дефинираме слой за вграждане с предварително обучени тегла:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Note**: Забележете, че задаваме `trainable=False`, когато създаваме `Embedding`, което означава, че няма да обучаваме отново слоя Embedding. Това може да доведе до леко намаляване на точността, но ускорява процеса на обучение.

### Използване на речник за вграждане

Един проблем с предишния подход е, че речниците, използвани в TextVectorization и Embedding, са различни. За да преодолеем този проблем, можем да използваме едно от следните решения:
* Преобучаване на модела Word2Vec върху нашия речник.
* Зареждане на нашия набор от данни с речника от предварително обучения модел Word2Vec. Речниците, използвани за зареждане на набора от данни, могат да бъдат зададени по време на зареждането.

Вторият подход изглежда по-лесен, затова нека го приложим. На първо място, ще създадем слой `TextVectorization` със зададен речник, взет от вгражданията на Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Библиотеката за вграждане на думи gensim съдържа удобна функция, `get_keras_embeddings`, която автоматично ще създаде съответния слой за вграждане на Keras за вас.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Една от причините, поради които не наблюдаваме по-висока точност, е, че някои думи от нашия набор от данни липсват в предварително обучената GloVe лексика и следователно те са на практика игнорирани. За да преодолеем това, можем да обучим собствени вграждания, базирани на нашия набор от данни.


## Контекстуални вграждания

Едно от основните ограничения на традиционните предварително обучени представяния на вграждания като Word2Vec е фактът, че, въпреки че могат да уловят някакво значение на дадена дума, те не могат да различат различните ѝ значения. Това може да създаде проблеми в последващите модели.

Например, думата „play“ има различно значение в тези две изречения:
- Отидох на **пиеса** в театъра.
- Джон иска да **играе** с приятелите си.

Предварително обучените вграждания, за които говорихме, представят и двете значения на думата „play“ в едно и също вграждане. За да преодолеем това ограничение, трябва да изградим вграждания, базирани на **езиков модел**, който е обучен върху голям корпус от текст и *знае* как думите могат да се комбинират в различни контексти. Обсъждането на контекстуални вграждания е извън обхвата на този урок, но ще се върнем към тях, когато говорим за езикови модели в следващия модул.



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
